# Networks of Harry's House

In [1]:
#IMPORT

import tweepy
import CONFIG
import csv
import os
import json
import numpy as np
import pandas as pd
import time
import itertools
import networkx as nx

from collections import Counter

In [2]:
path_data = os.getcwd() + '\HARRY_HOUSE_DATA'
path_zoom_data = path_data + '\\21maggio_27maggio'
path_networks = os.getcwd() + '\\NETWORKS'
os.chdir(path_data)
print(os.getcwd())

C:\Users\sarac\OneDrive - Università degli Studi di Milano\UNI 2022\_SOCIALMEDIA\#HARRYSHOUSE\HARRY_HOUSE_DATA


In [5]:
os.chdir(path_zoom_data)
print(os.getcwd())

C:\Users\sarac\OneDrive - Università degli Studi di Milano\UNI 2022\_SOCIALMEDIA\#HARRYSHOUSE\HARRY_HOUSE_DATA\21maggio_27maggio


In [6]:
client = tweepy.Client(bearer_token = CONFIG.BEARER_TOKEN)

In [8]:
tweets_extended = json.load(open('tweets_extended_2105_2705.json'))
#authors = json.load(open('author_1705.json'))
hashtags = json.load(open('tot_hashtags_2105_2705.json'))

## Rete sugli Hashtags

Creando una rete sugli hashtags possiamo individuare quali sono i nodi che trattano degli stessi argomenti. 

Iniziamo con il creare la rete prendendo in considerazione tutti gli hashtag di interesse tranne su _#HarrysHouse_ su cui abbiamo basato la collezione dei dati.

In [9]:
#dizionario in cui le chiavi sono gli hashtage e ci associamo tutti coloro che l'hanno usato
#per ora è solo la struttura

hashtags_used_by_authors = {}
for h in hashtags.keys():
    #non inseriamo HarrysHouse
    if h.lower() != 'harryshouse':
        hashtags_used_by_authors[h.lower()] = []

In [10]:
len(hashtags_used_by_authors)

1669

In [11]:
#riempiamo il dizionario
for t in tweets_extended:
    try:
        tweet_entities = t['entities']
    except:
        continue

    list_hashtags = tweet_entities.get('hashtags') #prendo la lista di # usati dall'autore
    author_tweet = t['author_id'] #prendo l'id dell'autore
        
    if list_hashtags:
        for hashtag in list_hashtags:
            h = hashtag['tag'].lower()
            if h != 'harryshouse':
                try:
                    list_users = hashtags_used_by_authors[h]
                except:
                    #è possibile che non sia presente un # nel dict perchè ho tolto i # usati una sola volta
                    continue
                if author_tweet in list_users:
                    #se l'utente è gia stato inserito per un tweet passato non lo inserisco
                    continue
                else:
                    list_users.append(author_tweet)

In [12]:
hashtags_used_by_authors

{'harrystyles': ['1139485149232533504',
  '1108706756073525249',
  '1308376762821222402',
  '1169483880',
  '1427578285869830146',
  '591121832',
  '25381805',
  '3988832955',
  '2684654995',
  '1490499164215599106',
  '19132124',
  '1529031151129706497',
  '32995576',
  '596963902',
  '1077672583791869953',
  '3386739964',
  '1279761231423909889',
  '1308928031289937922',
  '214837137',
  '23462167',
  '1390673255644598276',
  '1180199618174423040',
  '1621599468',
  '1260164819468128256',
  '3304704213',
  '1374048963314249728',
  '1152127069289103360',
  '1214400357662691333',
  '1476791804745768961',
  '31537446',
  '1364217733685129216',
  '90036845',
  '1225675522148950016',
  '2606115335',
  '350603725',
  '1433265076677578753',
  '2731725010',
  '3303651021',
  '1460107617808703491',
  '1316492060023681026',
  '1277338966757453824',
  '2768935857',
  '1519250082171965441',
  '1510851109425926150',
  '110048949',
  '744215139805708288',
  '41096139',
  '1466157533706469389',
  '

In [13]:
with open('twitter_network_hashtags_harryshouse_small.csv','w', encoding="utf-8") as ft:
    #riga di intestazione nel file per fargli capire che le colonne sono id e peso (per gephi)
    ft.write('Source,Target,Weight\n')
    #per ogni coppia di account
    for a, b in itertools.combinations(hashtags_used_by_authors.keys(),2):        
        common = set(hashtags_used_by_authors[a]).intersection(set(hashtags_used_by_authors[b]))
        if len(common) > 1: #scrivo sul file solo le coppie che hanno più di un hashtag in comune
            ft.write(f'{a},{b},{len(common)}\n')   

In [14]:
common = 0
with open('twitter_network_hashtags_harryshouse_big.csv','w', encoding="utf-8") as ft:
    #riga di intestazione nel file per fargli capire che le colonne sono id e peso (per gephi)
    ft.write('Source,Target,Weight\n')
    #per ogni coppia di account
    for a, b in itertools.combinations(hashtags_used_by_authors.keys(),2):        
        common = set(hashtags_used_by_authors[a]).intersection(set(hashtags_used_by_authors[b]))
        ft.write(f'{a},{b},{len(common)}\n')  

### Costruzione del grafo in estensione .gexf

In [15]:
network= []
count = 0

with open('twitter_network_hashtags_harryshouse_small.csv', 'r', encoding="utf8") as file:
    reader = csv.reader(file)
    
    for row in reader:
        if count == 0: #salto la riga di intestazioene
            count += 1
            continue
        network.append(row)

In [16]:
#cambio cartella per salvare file

path_networks_data = path_networks + '\\21maggio_27maggio'

isExist = os.path.exists(path_networks_data)

if not isExist:
    os.makedirs(path_networks_data)

os.chdir(path_networks_data)
print(os.getcwd())

C:\Users\sarac\OneDrive - Università degli Studi di Milano\UNI 2022\_SOCIALMEDIA\#HARRYSHOUSE\NETWORKS\21maggio_27maggio


In [17]:
g = nx.DiGraph()

for l in network:
    source = l[0]
    dest = l[1]
    w = l[2]
    g.add_edge(source, dest, weight = w)

nx.write_gexf(g, "small_network.gexf")

### Visualizzazione degli hashtags

In [18]:
count_hashtags = []

for l in hashtags_used_by_authors:
    count_tot = len(hashtags_used_by_authors[l])
    l_values = [l, count_tot]
    count_hashtags.append(l_values)

In [19]:
# take second element for sort
def takeSecond(elem):
    return elem[1]

# sort list with key
count_hashtags.sort(key=takeSecond, reverse=True)

In [20]:
with open('tot_hashtags.csv','w', encoding="utf-8", newline="") as f:
    header = ['Hashtag, Tot']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(count_hashtags[:30])

## Rete dei retweet

In [10]:
accounts2retweeters = {}
for a in authors.keys():
    accounts2retweeters[a] = []

In [12]:
for i, t in enumerate(tweets_extended):
    #prendiamo l'id dell'autore del tweet
    account = t['author_id'] 
    #richiedo coloro che hanno retweettato un tweet
    
    if t['public_metrics']['retweet_count'] > 0:
        users = client.get_retweeters(id = t['id']).data
        if users:
            for user in users:
                accounts2retweeters[account].append(user.data['id'])
        time.sleep(13)
    print(i, end=' ')

json.dump(accounts2retweeters, open('retweeters.json', 'w'))

    




0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 

KeyboardInterrupt: 